# TMDB

We have to breakdown to multiple files for a group of id, or else memory is not enough.


In [3]:
import tmdbsimple as tmdb
tmdb.API_KEY = "71e259894a515060876bab2a33d6bdc9"

In [2]:
from imdb import IMDb
import pandas as pd
from PIL import Image
from StringIO import StringIO
import requests
import os
import time
from shutil import copyfile
import csv

In [3]:
dir_python_notebook = os.getcwd()
dir_movie_project = os.path.abspath(os.path.join(dir_python_notebook, os.pardir))
dir_data = os.path.join(dir_movie_project, 'data')

# Load data

In [26]:
# get column names to retrieve information
tmdb_movie_id = tmdb.Movies(2)
movie_info = tmdb_movie_id.info()
tmdb_info_column  = tmdb_movie_id.info().keys()
tmdb_info_column  = [str(c) for c in tmdb_info_column]
tmdb_info_column.sort()

In [27]:
latest_r = requests.get('https://api.themoviedb.org/3/movie/latest?api_key=71e259894a515060876bab2a33d6bdc9')
latest_id = latest_r.json()['id']
latest_id

451100

In [46]:
id_start = 220001
id_range = 20000
id_end = id_start + id_range -1
print(id_end)
i_list = range(id_start,id_end+1)
print(i_list[-1])

240000
240000


In [ ]:
# if we already have a movie data file, we can just continue appending it instead of starting from id 1 again.
while id_start < latest_id:
    id_end = id_start + id_range - 1
    i_list = range(id_start,id_end + 1)
    
    movies = pd.DataFrame(columns=tmdb_info_column)
    new_filename = str(dir_data)+'\\drv_tmdb_movie_details_'+str(id_start)+'_'+str(id_end)+'.csv'

    if os.path.isfile(new_filename):
        movies = pd.read_csv(new_filename,index_col=0, sep='\t', encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)
        if len(movies['id']) >0:        
            movie_ids = movies['id'].tolist()
            tmdb_info_column = movies.columns
            i_list = [x for x in i_list if x not in movie_ids]
            movies = pd.DataFrame(columns=tmdb_info_column)
    else:
        movies.to_csv(new_filename,header =tmdb_info_column, sep='\t', encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)

    for i in i_list:
        #skip the non-existing movie ids
        if (i % 40 == 0):
            # make sure we do not hit API limit
            time.sleep(12) 
            movies.to_csv(new_filename,mode = 'a',header=False, sep='\t', encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)
            movies = pd.DataFrame(columns=tmdb_info_column)
        try:
            tmdb_movies = tmdb.Movies(i)
            info = tmdb_movies.info()

            movie_details = []
            for c in tmdb_info_column:
                if info.has_key(c):
                    if info[c] is not None:
                        if c in ['genres','spoken_languages','production_countries','production_companies']:
                            movie_details.append([d['name'] for d in info[c]])
                        elif c in ['belongs_to_collection']:
                            movie_details.append(info[c]['name'])
                        else:
                            movie_details.append(info[c]) 
                    else:
                        movie_details.append(None)
                else:
                        movie_details.append(None)
            movies.loc[i] = movie_details
        except Exception:
            continue

    movies.to_csv(new_filename,mode = 'a',header=False, sep='\t', encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)
    movies = pd.DataFrame(columns=tmdb_info_column)
    id_start = id_start + id_range              

In [34]:

id_start = 200001
id_range = 20000

while id_start < 220000:
    id_end = id_start + id_range - 1
    filename_json = str(dir_data)+'\\drv_tmdb_movie_details_'+str(id_start)+'_'+str(id_end)+'.json'
    filename_csv = str(dir_data)+'\\drv_tmdb_movie_details_'+str(id_start)+'_'+str(id_end)+'.csv'
    if os.path.isfile(filename_json):
        movies = pd.read_json(filename_json)
        tmdb_info_column = movies.columns
        movies.to_csv(filename_csv,header =tmdb_info_column, sep='\t', encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)
    id_start = id_start + id_range

In [5]:
tmdb_filename = str(dir_data)+'\\drv_tmdb_movie_details_280001_300000.csv'
tmdb_movies = pd.read_csv(tmdb_filename,index_col=0, sep='\t', encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)

In [6]:
max(tmdb_movies['id'])

300000.0